Import things


In [3]:
import torch
import torchvision
import einops

from torch import nn
from IPython.display import display

pil = torchvision.transforms.ToPILImage()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


trying to use einops to split image into patches


In [4]:
mnist = torchvision.datasets.MNIST("dataset", download=True)
image = mnist.data[2]

display(pil(image))
#该函数的作用就是把Tensor数据变为完整原始的图片数据（保存后可以直接双击打开的那种）
display(image.shape)

result = einops.rearrange(
    image.reshape((28, 28)), "(pw w) (ph h) -> (pw ph) w h", pw=2, ph=2
)
#重新指定维度，使得一张图片切成四块


for i in range(4):
    display(pil(result[i]))

# in the following code we use train_set or test_set instead of mnist.
del mnist


100.0%


Extracting dataset\MNIST\raw\train-images-idx3-ubyte.gz to dataset\MNIST\raw



102.8%


Extracting dataset\MNIST\raw\train-labels-idx1-ubyte.gz to dataset\MNIST\raw



100.0%


Extracting dataset\MNIST\raw\t10k-images-idx3-ubyte.gz to dataset\MNIST\raw



112.7%
D:\annaconda\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting dataset\MNIST\raw\t10k-labels-idx1-ubyte.gz to dataset\MNIST\raw



torch.Size([28, 28])

ok let's try other things with einops


In [5]:
from einops import rearrange

a = torch.rand((1, 28, 28))

rearrange(a, "b (pw w) (ph h) -> b (pw ph) (w h)", pw=4, ph=4).shape


torch.Size([1, 16, 49])

I got a problem: torch.cat doesn't work the same way as tensor addition. so we need to manually repeat the tensor in other channels.


In [6]:
a = torch.Tensor(((1, 1), (2, 2), (3, 3)))
print(a)
a = a.reshape((1, 3, 2))
print(einops.repeat(a, "() h w -> repeat h w", repeat=3))


tensor([[1., 1.],
        [2., 2.],
        [3., 3.]])
tensor([[[1., 1.],
         [2., 2.],
         [3., 3.]],

        [[1., 1.],
         [2., 2.],
         [3., 3.]],

        [[1., 1.],
         [2., 2.],
         [3., 3.]]])


looks good to me

now, let's try to create a vision transformer


but before that, let's try to figure out how pytorch handles tensor addition


In [7]:
a = torch.Tensor(((1, 2), (3, 4), (5, 6)))
print(a)
b = torch.Tensor((1, 2))
print(b)

print(a + b)


tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])
tensor([1., 2.])
tensor([[2., 4.],
        [4., 6.],
        [6., 8.]])


ok so it worked as i expected.


In [8]:
a = (1, 2, 3)

b, c, d = a

print(b, c, d)


1 2 3


first, we need to load mnist data:


In [9]:
from torchvision import transforms

#torchvision.transforms是常见的图像预处理方法

#transforms.Compose串联多个图片变换的操作
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]   #第一步，转为tenser类；第二个操作，归一化
)

train_set = torchvision.datasets.MNIST(
    "dataset", train=True, download=True, transform=transform
)
test_set = torchvision.datasets.MNIST("dataset", train=False, transform=transform)


then, we define the network


In [10]:
from einops.layers.torch import Rearrange


class VisionTransformer(nn.Module):
    def __init__(self):
        super().__init__()

        # shape: (batch, 28, 28) == (batch, 4*7, 4*7) -> (batch, 4*4, 7*7)
        
            #nn.Sequential 是一个时序容器，
        self.patch_embedding = nn.Sequential(
            Rearrange("b (pw w) (ph h) -> b (pw ph) (w h)", pw=4, ph=4),
            nn.Linear(7 * 7, 7 * 7),
        )
        
        #有重复part的分块
        
        # shape: (batch, 4*4+1, 7*7)
        self.class_token = nn.parameter.Parameter(torch.randn((1, 1, 7 * 7)))
        # shape: same
        self.position_encodings = nn.parameter.Parameter(
            torch.randn((1, 4 * 4 + 1, 7 * 7))
        )
        # the actual transformer, we use the transformer provided by pytorch here.
        # d_model is the dimention for the embedding vector
        # n_head is the number of heads in the multi head self attention thing
        # d_model has to be divisible by n_head
        # dim_feedforward is the number of hidden nodes in hidden layer in feedforward block
        # batch_first means the first dim of the tensor is batch
        # shape: same
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=7 * 7, nhead=7, dim_feedforward=128, batch_first=True
            ),
            3,
        )
        # the final MLP
        # shape: (batch, 4*4+1, 7*7) -> (batch, 10)
        self.mlp_head = nn.Sequential(nn.LayerNorm(7 * 7), nn.Linear(7 * 7, 10))

    def forward(self, x):
        x = self.patch_embedding(x)
        batch_size, _, _ = x.shape
        class_token = einops.repeat(
            self.class_token,
            "() words features -> repeat words features",
            repeat=batch_size,
        )
        x = torch.cat((class_token, x), dim=-2)
        x += self.position_encodings
        x = self.transformer_encoder(x)
        x = x[:, 0]
        x = self.mlp_head(x)
        return x


let's try to use this absolute abomination we've created


In [11]:
vision_transformer_model = VisionTransformer()
vision_transformer_model(torch.rand((1, 28, 28)))


tensor([[ 0.4396, -0.6882, -0.5664,  0.1234, -0.4082, -0.2944,  0.5940, -0.0736,
         -0.0883, -0.5515]], grad_fn=<AddmmBackward>)

ok, it works. And the shape seems to be alright.

now let's try to train it.


In [12]:
criterion = nn.CrossEntropyLoss()
epochs = 4
batch_size = 256
optimizer = torch.optim.Adam(vision_transformer_model.parameters(), lr=0.001)
# for every step, this scheduler times the lr by gamma
# step_size if how many steps before it times the lr by gamma
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)

vision_transformer_model.to(device)
vision_transformer_model.train()
for epoch in range(epochs):
    for batch_idx, (x, y) in enumerate(train_loader):
        x = einops.rearrange(x, "b c h w -> (b c) h w")
        optimizer.zero_grad()
        output = vision_transformer_model(x.to(device))
        loss = criterion(output, y.to(device))
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(
                f"epoch: {epoch}, i: {batch_idx*len(x)}/{len(train_loader.dataset)}, loss: {loss.item()}"
            )

    scheduler.step()


epoch: 0, i: 0/60000, loss: 2.3846499919891357
epoch: 0, i: 2560/60000, loss: 2.137324094772339
epoch: 0, i: 5120/60000, loss: 1.6387113332748413
epoch: 0, i: 7680/60000, loss: 1.3975093364715576
epoch: 0, i: 10240/60000, loss: 0.9649822115898132
epoch: 0, i: 12800/60000, loss: 0.9680061340332031
epoch: 0, i: 15360/60000, loss: 0.7219030857086182
epoch: 0, i: 17920/60000, loss: 0.5770267248153687
epoch: 0, i: 20480/60000, loss: 0.5623785257339478
epoch: 0, i: 23040/60000, loss: 0.5560121536254883
epoch: 0, i: 25600/60000, loss: 0.44870585203170776
epoch: 0, i: 28160/60000, loss: 0.5034988522529602
epoch: 0, i: 30720/60000, loss: 0.47477445006370544
epoch: 0, i: 33280/60000, loss: 0.4478832483291626
epoch: 0, i: 35840/60000, loss: 0.35157737135887146
epoch: 0, i: 38400/60000, loss: 0.3507702648639679
epoch: 0, i: 40960/60000, loss: 0.3347446322441101
epoch: 0, i: 43520/60000, loss: 0.2557900846004486
epoch: 0, i: 46080/60000, loss: 0.41349756717681885
epoch: 0, i: 48640/60000, loss: 0.2

looks like a great success to me

( actually, as i was creating this notebook, many things went wrong. But thanks to the magic of _editing_, everything seems to be fine :) )


In [13]:
batch_size = 256
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

vision_transformer_model.to(device)
vision_transformer_model.eval()
correct_count = 0
for batch_idx, (x, y) in enumerate(test_loader):
    x = einops.rearrange(x, "b c h w -> (b c) h w")
    output = vision_transformer_model(x.to(device))
    correct_count += output.argmax(dim=-1).eq(y.to(device)).count_nonzero()

    if batch_idx % 10 == 0:
        print(f"i: {batch_idx*len(x)}/{len(test_loader.dataset)}")

print(
    f"correct: {correct_count}/{len(test_loader.dataset)}, {correct_count/len(test_loader.dataset)*100:.2f}%"
)


i: 0/10000
i: 2560/10000
i: 5120/10000
i: 7680/10000
correct: 9637/10000, 96.37%


let's compare the model with a regular CNN model:


In [14]:
cnn_model = nn.Sequential(
    nn.Conv2d(
        in_channels=1,
        out_channels=16,
        kernel_size=5,
        stride=1,
        padding="same",
    ),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(16, 32, 5, 1, "same"),
    nn.ReLU(),
    nn.MaxPool2d(2),
    # defaults: start_dim: 1, end_dim: -1
    nn.Flatten(),
    nn.Linear(32 * 7 * 7, 10),
)
cnn_model(torch.randn((1, 1, 28, 28)))


tensor([[ 0.0458,  0.1063, -0.0766,  0.0239, -0.0795, -0.1176,  0.0239,  0.0858,
         -0.0522,  0.1815]], grad_fn=<AddmmBackward>)

In [15]:
criterion = nn.CrossEntropyLoss()
epochs = 4
batch_size = 256
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)
# for every step, this scheduler times the lr by gamma
# step_size if how many steps before it times the lr by gamma
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)

cnn_model.to(device)
cnn_model.train()
for epoch in range(epochs):
    for batch_idx, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        output = cnn_model(x.to(device))
        loss = criterion(output, y.to(device))
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(
                f"epoch: {epoch}, i: {batch_idx*len(x)}/{len(train_loader.dataset)}, loss: {loss.item()}"
            )

    scheduler.step()


epoch: 0, i: 0/60000, loss: 2.302013397216797
epoch: 0, i: 2560/60000, loss: 1.0947916507720947
epoch: 0, i: 5120/60000, loss: 0.5394686460494995
epoch: 0, i: 7680/60000, loss: 0.49899035692214966
epoch: 0, i: 10240/60000, loss: 0.3036370873451233
epoch: 0, i: 12800/60000, loss: 0.39212530851364136
epoch: 0, i: 15360/60000, loss: 0.23711681365966797
epoch: 0, i: 17920/60000, loss: 0.17943309247493744
epoch: 0, i: 20480/60000, loss: 0.26187723875045776
epoch: 0, i: 23040/60000, loss: 0.16979683935642242
epoch: 0, i: 25600/60000, loss: 0.18883703649044037
epoch: 0, i: 28160/60000, loss: 0.20958547294139862
epoch: 0, i: 30720/60000, loss: 0.2108856588602066
epoch: 0, i: 33280/60000, loss: 0.1452208012342453
epoch: 0, i: 35840/60000, loss: 0.16448284685611725
epoch: 0, i: 38400/60000, loss: 0.12642329931259155
epoch: 0, i: 40960/60000, loss: 0.13525404036045074
epoch: 0, i: 43520/60000, loss: 0.1044606938958168
epoch: 0, i: 46080/60000, loss: 0.21315431594848633
epoch: 0, i: 48640/60000, l

In [16]:
a = torch.Tensor([[1, 2, 3], [1, 2, 1], [1, 2, 1]])
b = torch.Tensor([2, 1, 0])
a.argmax(dim=-1).eq(b).count_nonzero()


tensor(2)

In [17]:
batch_size = 256
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

cnn_model.to(device)
cnn_model.eval()
correct_count = 0
for batch_idx, (x, y) in enumerate(test_loader):
    output = cnn_model(x.to(device))
    correct_count += output.argmax(dim=-1).eq(y.to(device)).count_nonzero()

    if batch_idx % 10 == 0:
        print(f"i: {batch_idx*len(x)}/{len(test_loader.dataset)}")

print(
    f"correct: {correct_count}/{len(test_loader.dataset)}, {correct_count/len(test_loader.dataset)*100:.2f}%"
)


i: 0/10000
i: 2560/10000
i: 5120/10000
i: 7680/10000
correct: 9844/10000, 98.44%
